# Vitessce Widget Tutorial

# Visualization of a SpatialData object

## Import dependencies


In [ ]:
import os
from os.path import join, isfile, isdir
from urllib.request import urlretrieve
import zipfile

from vitessce import (
    VitessceConfig,
    ViewType as vt,
    CoordinationType as ct,
    CoordinationLevel as CL,
    SpatialDataWrapper,
    get_initial_coordination_scope_prefix
)

In [ ]:
data_dir = "data"
zip_filepath = join(data_dir, "visium_hd_3.0.0_io.zip")
spatialdata_filepath = join(data_dir, "visium_hd_3.0.0.spatialdata.zarr")

In [ ]:
if not isdir(spatialdata_filepath):
    if not isfile(zip_filepath):
        os.makedirs(data_dir, exist_ok=True)
        urlretrieve('https://s3.embl.de/spatialdata/spatialdata-sandbox/visium_hd_3.0.0_io.zip', zip_filepath)
    with zipfile.ZipFile(zip_filepath,"r") as zip_ref:
        zip_ref.extractall(data_dir)
        os.rename(join(data_dir, "data.zarr"), spatialdata_filepath)

## Rasterize bins
Reference: https://spatialdata.scverse.org/en/stable/tutorials/notebooks/notebooks/examples/technology_visium_hd.html#performant-on-the-fly-data-rasterization

In [ ]:
from spatialdata import (
    read_zarr,
    rasterize_bins,
    rasterize_bins_link_table_to_labels
)

In [ ]:
sdata = read_zarr(spatialdata_filepath)
sdata

In [ ]:
for bin_size in ["016", "008", "002"]:
    # rasterize_bins() requires a compresed sparse column (csc) matrix
    sdata.tables[f"square_{bin_size}um"].X = sdata.tables[f"square_{bin_size}um"].X.tocsc()
    rasterized = rasterize_bins(
        sdata,
        f"Visium_HD_Mouse_Small_Intestine_square_{bin_size}um",
        f"square_{bin_size}um",
        "array_col",
        "array_row",
        # We want to rasterize to a Labels element, rather than an Image element.
        return_region_as_labels=True
    )
    sdata[f"rasterized_{bin_size}um"] = rasterized
    rasterize_bins_link_table_to_labels(
        sdata,
        table_name=f"square_{bin_size}um",
        rasterized_labels_name=f"rasterized_{bin_size}um",
    )
    sdata.write_element(f"rasterized_{bin_size}um")

In [ ]:
sdata

In [ ]:
# Test with spatialdata-plot

In [ ]:
import spatialdata_plot

In [ ]:
sdata.pl.render_images("Visium_HD_Mouse_Small_Intestine_full_image").pl.render_labels("rasterized_016um").pl.show("Visium_HD_Mouse_Small_Intestine")

## Configure Vitessce

Vitessce needs to know which pieces of data we are interested in visualizing, the visualization types we would like to use, and how we want to coordinate (or link) the views.

In [ ]:
vc = VitessceConfig(
    schema_version="1.0.18",
    name='Visium HD SpatialData Demo',
)
# Add data to the configuration:
wrapper = SpatialDataWrapper(
    sdata_path=spatialdata_filepath,
    # The following paths are relative to the root of the SpatialData zarr store on-disk.
    image_path="images/Visium_HD_Mouse_Small_Intestine_full_image",
    table_path="tables/square_016um",
    obs_feature_matrix_path="tables/square_016um/X",
    obs_segmentations_path="labels/rasterized_016um",
    #region="CytAssist_FFPE_Human_Breast_Cancer",
    coordinate_system="Visium_HD_Mouse_Small_Intestine",
    coordination_values={
        # The following tells Vitessce to consider each observation as a "bin"
        "obsType": "bin",
    }
)
dataset = vc.add_dataset(name='Visium HD').add_object(wrapper)

# Add views (visualizations) to the configuration:
spatial = vc.add_view("spatialBeta", dataset=dataset)
feature_list = vc.add_view("featureList", dataset=dataset)
layer_controller = vc.add_view("layerControllerBeta", dataset=dataset)
vc.link_views_by_dict([spatial, layer_controller], {
    'imageLayer': CL([{
        'photometricInterpretation': 'RGB',
    }]),
}, scope_prefix=get_initial_coordination_scope_prefix("A", "image"))
obs_sets = vc.add_view(vt.OBS_SETS, dataset=dataset)
vc.link_views([spatial, layer_controller, feature_list, obs_sets], ['obsType'], [wrapper.obs_type_label])

# Layout the views
vc.layout(spatial | (feature_list / layer_controller / obs_sets));

### Render the widget

In [ ]:
vw = vc.widget()
vw

In [ ]:
import json
print(json.dumps(vc.to_dict(base_url=f"http://localhost:{vw.port}")))